# Class Scheduling Problem
### Schedule the timetable for a school based on the timetable skeleton provided.
## Overview
The idea is to take a timetable schedule for a school (i.e. period times in a week) and populate it with lessons based on what classes students take, what classes teachers teach, venues, etc.

This is quite a complex problem to do by hand, so I am hoping that it can be solved with optimization.

### Data Gathering

In [97]:
import pandas as pd
import numpy as np

In [63]:
students_df = pd.read_csv("data/students.csv")
students_df

,Student_ID,Name,Grade,Subject,Set
0,1,Colleen,10,English,1
1,1,Colleen,10,Mathematics,1
2,1,Colleen,10,Science,1
3,1,Colleen,10,Geography,1
4,1,Colleen,10,Drama,1
5,2,Tess,10,English,1
6,2,Tess,10,Mathematics,1
7,2,Tess,10,Science,1
8,2,Tess,10,Geography,1
9,2,Tess,10,Drama,1


In [64]:
lessons_df = pd.read_csv("data/lessons_sml.csv")
lessons_df

,Teacher ID,Teacher,Subject,Grade,Set,Hours per Week
0,1,Chris,English,10,1,4
1,2,Nick,Mathematics,10,1,5
2,3,Ryan,Science,10,1,5
3,4,Chloe,Geography,10,1,4
4,5,Jaxi,Drama,10,1,2
5,6,Smith,Economics,10,1,3


In [65]:
venues_df = pd.read_csv("data/venues.csv")
venues_df

,Venue ID,Name,Capacity
0,1,LT1,10
1,2,LT2,10
2,3,LT3,10
3,4,LT4,10
4,5,LT5,10


In [66]:
timetable_df = pd.read_csv("data/timetable.csv")
timetable_df

,Day,Period Name,Start time,Duration (hours)
0,Monday,assembly,08:00,1
1,Monday,class,09:00,1
2,Monday,class,10:00,1
3,Monday,break,11:00,1
4,Monday,class,12:00,1
5,Monday,class,13:00,1
6,Monday,lunch,14:00,1
7,Tuesday,assembly,08:00,1
8,Tuesday,class,09:00,1
9,Tuesday,class,10:00,1


### Data Pre-Processing

In [67]:
lessons_per_day = []
days_per_week = range(5)
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

for day in days:
    lessons_per_day.append(timetable_df.groupby('Day')['Period Name'].value_counts()[day]['class'])

lessons_per_day

[4, 4, 4, 4, 4]

In [68]:
students_per_venue = []
number_of_venues = range(venues_df.count()['Venue ID'])

for i in number_of_venues:
    students_per_venue.append(venues_df["Capacity"][i])
    
students_per_venue

[10, 10, 10, 10, 10]

In [69]:
subjects = students_df["Subject"].unique()
subjects_check = lessons_df["Subject"].unique()

if subjects_check.shape != subjects.shape:
    print("Something is wrong. Number of teacher subjects different to number of student subjects.")
else:
    number_of_subjects = range(subjects.shape[0])
    print("There are {} subjects.".format(len(number_of_subjects)))
print(subjects)

subject_hours_per_week = lessons_df["Hours per Week"]
print(subject_hours_per_week) 

There are 6 subjects.
['English' 'Mathematics' 'Science' 'Geography' 'Drama' 'Economics']
0    4
1    5
2    5
3    4
4    2
5    3
Name: Hours per Week, dtype: int64


In [70]:
number_of_students = range(students_df["Student_ID"].unique().shape[0])
len(number_of_students)

4

In [211]:
student_ids = students_df["Student_ID"].unique()
student_enrollment = []

for ID in student_ids:
    enrollment = students_df[students_df["Student_ID"] == ID]["Subject"].unique()
    student_enrollment.append(enrollment)

student_enrollment


[array(['English', 'Mathematics', 'Science', 'Geography', 'Drama'],
       dtype=object),
 array(['English', 'Mathematics', 'Science', 'Geography', 'Drama'],
       dtype=object),
 array(['English', 'Mathematics', 'Science', 'Geography', 'Drama'],
       dtype=object),
 array(['Economics'], dtype=object)]

In [212]:
student_enrollment_mask = np.zeros((len(number_of_students),len(number_of_subjects)),dtype=np.uint8)

subject_masks = np.diag(np.ones(len(subjects),dtype=np.uint8))

for i,student in enumerate(student_enrollment):
    for k,subject in enumerate(subjects):
        if subject in student:
            student_enrollment_mask[i] = np.bitwise_or(student_enrollment_mask[i],subject_masks[k])
    
student_enrollment_mask


array([[1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 1]], dtype=uint8)

### Problem Formulation & Optimization

In [213]:
from mip import Model, xsum, minimize, INTEGER, BINARY, OptimizationStatus

m = Model()

# Timetable
# days_per_week -> lessons_per_day -> subjects -> students
x_timetable = [[[m.add_var(var_type=BINARY) 
                   for j in number_of_subjects] 
                   for i in range(lessons_per_day[k])] 
                   for k in days_per_week]

# z = [[m.add_var() 
#       for j in number_of_subjects]
#       for k in days_per_week]


In [214]:
### CONSTRAINTS

# hours per subject must add up over week
for j in number_of_subjects:
    m += xsum(x_timetable[k][i][j] for k in days_per_week for i in range(lessons_per_day[k])) == subject_hours_per_week[j]
        
# same subject cannot be repeated in a day
for k in days_per_week:
    for j in number_of_subjects:
        m += xsum(x_timetable[k][i][j] for i in range(lessons_per_day[k])) <= 1

# classes cannot happen in same period if students are enrolled for them
for k in days_per_week:
    for i in range(lessons_per_day[k]):
        for student in student_enrollment_mask:
            m += xsum(x_timetable[k][i][j]*student[j] for j in number_of_subjects) <= 1
            
# scheduling_mask = np.arange(1,lessons_per_day[0]+1)            
# for j in number_of_subjects:
#     for k in days_per_week:
#         m += z[k][j] == xsum(x_timetable[0][i][j]*scheduling_mask[i] for i in range(lessons_per_day[0])) - xsum(x_timetable[k][i][j]*scheduling_mask[i] for i in range(lessons_per_day[k])) 
    
# for j in number_of_subjects:
#     m += z[j] >= 0

In [215]:
# COST

# get periods at a similar time during the day
# m.objective = minimize(xsum(z[j] for j in number_of_subjects))

In [216]:
status = m.optimize()

if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
elif status == OptimizationStatus.INFEASIBLE:
    print('Problem is infeasible')

optimal solution cost 0.0 found


### Data Consolidation

In [217]:
results_x = np.array(x_timetable)
results = np.zeros_like(results_x.flat)

for n, el in enumerate(results_x.flat):
    results[n] = el.x

results = results.reshape(results_x.shape)

In [218]:
print(subjects)
for k in days_per_week:
    print()
    print(days[k]+":")
    
    for i in range(lessons_per_day[k]):
        print("Period "+str(i)+" ",end="")
        print(results[k,i])

['English' 'Mathematics' 'Science' 'Geography' 'Drama' 'Economics']

Monday:
Period 0 [0.0 0.0 1.0 0.0 0.0 1.0]
Period 1 [1.0 0.0 0.0 0.0 0.0 0.0]
Period 2 [0.0 1.0 0.0 0.0 0.0 0.0]
Period 3 [0.0 0.0 0.0 1.0 0.0 0.0]

Tuesday:
Period 0 [0.0 0.0 0.0 1.0 0.0 1.0]
Period 1 [1.0 0.0 0.0 0.0 0.0 0.0]
Period 2 [0.0 0.0 1.0 0.0 0.0 0.0]
Period 3 [0.0 1.0 0.0 0.0 0.0 0.0]

Wednesday:
Period 0 [0.0 0.0 1.0 0.0 0.0 0.0]
Period 1 [0.0 1.0 0.0 0.0 0.0 0.0]
Period 2 [0.0 0.0 0.0 1.0 0.0 0.0]
Period 3 [1.0 0.0 0.0 0.0 0.0 0.0]

Thursday:
Period 0 [1.0 0.0 0.0 0.0 0.0 0.0]
Period 1 [0.0 1.0 0.0 0.0 0.0 0.0]
Period 2 [0.0 0.0 0.0 0.0 1.0 0.0]
Period 3 [0.0 0.0 1.0 0.0 0.0 0.0]

Friday:
Period 0 [0.0 0.0 0.0 1.0 0.0 0.0]
Period 1 [0.0 0.0 0.0 0.0 1.0 1.0]
Period 2 [0.0 0.0 1.0 0.0 0.0 0.0]
Period 3 [0.0 1.0 0.0 0.0 0.0 0.0]
